In this notebook, we will be generating features to use for PIPE.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [48]:
masks_path = "data/yeast_masks_singlesite_domains_only_filtered.pkl"
uniprot_file = "data/uniprot-proteome UP000002311.tab"# reference proteome at https://www.uniprot.org/proteomes/UP000002311
pssm_path = "data/yeast_pssms/"

# Load data and masks

## Load masks

In [49]:
ppi_masks = pd.read_pickle(masks_path)
ppi_masks.head()

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [50]:
ppi_masks.shape

In [61]:
# All the protein sequences we need information for
proteins = ppi_masks['Uniprot ID A'].append(ppi_masks['Uniprot ID B'], ignore_index=True).unique()
proteins, proteins.shape

(array(['P53081', 'Q12188', 'Q03020', ..., 'P22219', 'P12612', 'P38080'],
       dtype=object), (1258,))

## Load sequences

In [51]:
uniprot_df = pd.read_csv(uniprot_file,
                        sep = "\t", index_col='Entry')
print("Loaded UniProt proteome")
uniprot_df

Loaded UniProt proteome


,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence
Entry,,,,,,,
P21192,ACE2_YEAST,reviewed,Metallothionein expression activator,ACE2 YLR131C L3123 L9606.10,Saccharomyces cerevisiae (strain ATCC 204508 /...,770,MDNVVDPWYINPSGFAKDTQDEEYVQHHDNVNPTIPPPDNYILNNE...
P46993,ASG7_YEAST,reviewed,Protein ASG7 (A-specific gene 7 protein),ASG7 YJL170C J0514,Saccharomyces cerevisiae (strain ATCC 204508 /...,209,MTTLASSIEHKTKHLAAPFENDENPWMKKYCCQCKSCKMSVPVQPW...
P47117,ARP3_YEAST,reviewed,Actin-related protein 3 (Actin-like protein AR...,ARP3 ACT4 YJR065C J1760,Saccharomyces cerevisiae (strain ATCC 204508 /...,449,MSYLNNPAVVMDNGTGLTKLGFAGNDSPSWVFPTAIATAAPSNTKK...
P22768,ASSY_YEAST,reviewed,Argininosuccinate synthase (EC 6.3.4.5) (Citru...,ARG1 YOL058W O1228,Saccharomyces cerevisiae (strain ATCC 204508 /...,420,MSKGKVCLAYSGGLDTSVILAWLLDQGYEVVAFMANVGQEEDFDAA...
P29311,BMH1_YEAST,reviewed,Protein BMH1,BMH1 YER177W,Saccharomyces cerevisiae (strain ATCC 204508 /...,267,MSTSREDSVYLAKLAEQAERYEEMVENMKTVASSGQELSVEERNLL...
...,...,...,...,...,...,...,...
P47049,UBX6_YEAST,reviewed,UBX domain-containing protein 6,UBX6 YJL048C J1164,Saccharomyces cerevisiae (strain ATCC 204508 /...,396,MYEMSGIDSLFHDRVVHDYSHTSEQVIVVYISSAAGDNSWLHQWFK...
P53142,VPS73_YEAST,reviewed,Vacuolar protein sorting-associated protein 73,VPS73 YGL104C G3090,Saccharomyces cerevisiae (strain ATCC 204508 /...,486,MNRILSSASLLSNVSMPRQNKHKITKALCYAIIVASIGSIQFGYHL...
Q05919,VPS38_YEAST,reviewed,Vacuolar protein sorting-associated protein 38,VPS38 VPL17 YLR360W L8039.11,Saccharomyces cerevisiae (strain ATCC 204508 /...,439,MKRFLLSRRQRHLRMICFHNISLFRANGDSKLIKEYGDGFIPCFFI...


## ProtDCal

Generated using https://protdcal.zmb.uni-due.de/pages/form.php

## PSSMs

psiblast -query 'uniprot-proteome UP000002311.fasta' -db ~/sysc4906/s/swissprot -inclusion_ethresh 0.001 -num_iterations 2 -out_ascii_pssm yeast_filtered.pssm -save_pssm_after_last_round

/home/wma/sysc4906/ncbi-blast-2.11.0+/bin

for file in *.fasta; do echo "./runPsiBlast.sh '$file'"; done >> blastall.sh

rename -n 's/sp\|(.*?)\|.*/$1.pssm/' *

In [52]:
# psiblast -query .\output_sequences.fasta -db nr -out yeast_filtered_psiblast_out -evalue 0.001 -num_iterations 3 -out_pssm yeast_filtered_pssm_checkpoint -out_ascii_pssm yeast_filtered_pssm

In [53]:
AA = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

In [62]:
print("Loading PSSMs")
PSSMs = {}
for swissprot_id in proteins:
    pssm_A = pd.read_csv(pssm_path + swissprot_id + ".pssm", sep = "\s+", skiprows=2).reset_index()[AA].dropna()
    PSSMs[swissprot_id] = pssm_A.to_numpy()

In [65]:
PSSM_masks = []
print("Generating PSSM masks")
for UA, UB, SM in zip(ppi_masks['Uniprot ID A'], ppi_masks['Uniprot ID B'], ppi_masks['Sites Masks']):
    try:
#         lengths = (uniprot_df['Length'].loc[UA], uniprot_df['Length'].loc[UB])
        
        # number according to position    
        seqA = PSSMs[UA]
        seqB = PSSMs[UB]
        
#         # normalize
#         seqA = np.divide(seqA, lengths[0])
#         seqB = np.divide(seqB, lengths[1])
        
        # Do a dot product for each 20 length vector along each position
        mask = np.matmul(seqA, seqB.T)
        
        assert SM.shape == mask.shape
        
        PSSM_masks.append(mask)
    except KeyError as inst:
        print(UA, UB)
        print(f"No uniprot entry found for protein {inst.args}")
        position_landscape.append(np.NaN)

In [66]:
PSSM_masks[:10]

In [67]:
ppi_masks['PSSM_masks'] = PSSM_masks
ppi_masks

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[118.0, -11.0, -15.0, 5.0, 33.0, 11.0, -8.0, ..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[174.0, 68.0, 113.0, 28.0, -10.0, 152.0, 14.0..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[81.0, 82.0, 6.0, -8.0, 43.0, 56.0, 8.0, -9.0..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[114.0, -7.0, -27.0, -40.0, -32.0, -22.0, -28..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[164.0, -40.0, 3.0, -38.0, -37.0, 120.0, -32...."
...,...,...,...,...,...,...,...,...
1601,Q12403,P54837,[PF01105],[PF01105],"[(23, 218)]","[(20, 206)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[148.0, 17.0, 28.0, 80.0, 33.0, 56.0, 43.0, 8..."
1602,Q06549,Q06549,[PF00383],[PF00383],"[(8, 115)]","[(8, 115)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[111.0, -20.0, 32.0, -3.0, -26.0, 134.0, -35...."
1603,Q12335,Q12335,[PF03358],[PF03358],"[(8, 148)]","[(8, 148)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[206.0, 18.0, 25.0, 140.0, 75.0, 141.0, 132.0..."
1604,P53725,P38792,[PF10175],[PF15985],"[(9, 186)]","[(197, 239)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[71.0, -10.0, -17.0, 61.0, 70.0, 10.0, 70.0, ..."


## Position

In [68]:
position_landscape = []
print("Generating position feature landscape")
for UA, UB, SM in zip(ppi_masks['Uniprot ID A'], ppi_masks['Uniprot ID B'], ppi_masks['Sites Masks']):
    try:
        lengths = (uniprot_df['Length'].loc[UA], uniprot_df['Length'].loc[UB])
        
        # number according to position    
        seqA = np.arange(1, lengths[0]+1)[np.newaxis]
        seqB = np.arange(1, lengths[1]+1)[np.newaxis]
        
        # normalize
        seqA = np.divide(seqA, lengths[0])
        seqB = np.divide(seqB, lengths[1])
        
        mask = np.matmul(seqA.T, seqB)
        
        assert SM.shape == mask.shape
        
        position_landscape.append(mask)
    except KeyError as inst:
        print(UA, UB)
        print(f"No uniprot entry found for protein {inst.args}")
        position_landscape.append(np.NaN)
        
position_landscape = np.asarray(position_landscape)

Generating position feature landscape


C:\Users\mawil\Anaconda3\envs\SYSC4906\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [69]:
position_landscape[:2]

array([array([[1.20563272e-05, 2.41126543e-05, 3.61689815e-05, ...,
        3.44810957e-03, 3.46016590e-03, 3.47222222e-03],
       [2.41126543e-05, 4.82253086e-05, 7.23379630e-05, ...,
        6.89621914e-03, 6.92033179e-03, 6.94444444e-03],
       [3.61689815e-05, 7.23379630e-05, 1.08506944e-04, ...,
        1.03443287e-02, 1.03804977e-02, 1.04166667e-02],
       ...,
       [3.44810957e-03, 6.89621914e-03, 1.03443287e-02, ...,
        9.86159336e-01, 9.89607446e-01, 9.93055556e-01],
       [3.46016590e-03, 6.92033179e-03, 1.03804977e-02, ...,
        9.89607446e-01, 9.93067612e-01, 9.96527778e-01],
       [3.47222222e-03, 6.94444444e-03, 1.04166667e-02, ...,
        9.93055556e-01, 9.96527778e-01, 1.00000000e+00]]),
       array([[1.19560019e-06, 2.39120038e-06, 3.58680057e-06, ...,
        1.46819703e-03, 1.46939264e-03, 1.47058824e-03],
       [2.39120038e-06, 4.78240077e-06, 7.17360115e-06, ...,
        2.93639407e-03, 2.93878527e-03, 2.94117647e-03],
       [3.58680057e-06, 7.17

In [70]:
ppi_masks['position_landscape'] = position_landscape
ppi_masks

,Uniprot ID A,Uniprot ID B,Domain_id_a,Domain_id_b,Domain positions A,Domain positions B,Sites Masks,PSSM_masks,position_landscape
0,P53081,P53081,[PF01784],[PF01784],"[(12, 275)]","[(12, 275)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[118.0, -11.0, -15.0, 5.0, 33.0, 11.0, -8.0, ...","[[1.2056327160493826e-05, 2.4112654320987653e-..."
1,Q12188,P47037,[PF04825],[PF02463],"[(11, 136)]","[(2, 1208)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[174.0, 68.0, 113.0, 28.0, -10.0, 152.0, 14.0...","[[1.1956001912960306e-06, 2.391200382592061e-0..."
2,Q03020,P25374,[PF01592],[PF00266],"[(34, 161)]","[(100, 462)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[81.0, 82.0, 6.0, -8.0, 43.0, 56.0, 8.0, -9.0...","[[1.2194378391561491e-05, 2.4388756783122982e-..."
3,P17065,P51996,[PF06428],[PF00071],"[(79, 170)]","[(15, 176)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[114.0, -7.0, -27.0, -40.0, -32.0, -22.0, -28...","[[5.934788543484196e-06, 1.1869577086968392e-0..."
4,P00546,P20437,[PF00069],[PF00134],"[(8, 295)]","[(41, 193)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[164.0, -40.0, 3.0, -38.0, -37.0, 120.0, -32....","[[6.145979300341716e-06, 1.2291958600683432e-0..."
...,...,...,...,...,...,...,...,...,...
1601,Q12403,P54837,[PF01105],[PF01105],"[(23, 218)]","[(20, 206)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[148.0, 17.0, 28.0, 80.0, 33.0, 56.0, 43.0, 8...","[[2.1063717746182202e-05, 4.2127435492364404e-..."
1602,Q06549,Q06549,[PF00383],[PF00383],"[(8, 115)]","[(8, 115)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[111.0, -20.0, 32.0, -3.0, -26.0, 134.0, -35....","[[4.959333465582226e-05, 9.918666931164451e-05..."
1603,Q12335,Q12335,[PF03358],[PF03358],"[(8, 148)]","[(8, 148)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[206.0, 18.0, 25.0, 140.0, 75.0, 141.0, 132.0...","[[2.5507601265177026e-05, 5.101520253035405e-0..."
1604,P53725,P38792,[PF10175],[PF15985],"[(9, 186)]","[(197, 239)]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[71.0, -10.0, -17.0, 61.0, 70.0, 10.0, 70.0, ...","[[1.4975888819001409e-05, 2.9951777638002818e-..."
